# Rankings (top, controversial, pearls) @ Anime Expo

Hi! This is an overview of the main filters of Mangaki (top(), controversial(), random()), and how they could and should be improved.

It was presented at the **Anime & Manga Studies Symposium of Anime Expo** on July 2, 2017.

## Most popular

In [1]:
import pandas as pd

pd.DataFrame(list(Work.objects.popular().values('title', 'nb_ratings')[:10]))

,nb_ratings,title
0,1489,Death Note
1,1367,L'Attaque des Titans
2,1227,Le Voyage de Chihiro
3,1194,Naruto
4,1188,Princesse Mononoké
5,1167,Sword Art Online
6,1164,Fullmetal Alchemist: Brotherhood
7,1047,Fullmetal Alchemist
8,1009,Naruto: Shippuuden
9,996,Fairy Tail


## Top

In [23]:
import pandas as pd

pd.DataFrame(list(Work.objects.top().values('title', 'nb_likes', 'nb_dislikes', 'category__slug')[:10]))

,category__slug,nb_dislikes,nb_likes,title
0,manga,7,164,Fullmetal Alchemist - Edition reliée
1,manga,14,457,FullMetal Alchemist
2,anime,34,837,Fullmetal Alchemist: Brotherhood
3,anime,30,442,Steins;Gate
4,anime,31,860,Princesse Mononoké
5,anime,29,938,Le Voyage de Chihiro
6,anime,65,971,L'Attaque des Titans
7,manga,20,210,Tokyo ghoul
8,manga,26,480,Death note
9,anime,18,255,Your Lie in April


## Worst

In [30]:
import pandas as pd

pd.DataFrame(list(Work.objects.top().reverse().values('title', 'nb_likes', 'nb_dislikes', 'category__slug')[:10]))

,category__slug,nb_dislikes,nb_likes,title
0,anime,39,26,Girls Bravo: First Season
1,anime,38,32,Astarotte's Toy
2,anime,35,24,Captain Earth
3,anime,22,13,Vividred Operation
4,anime,33,27,Choujigen Game Neptune: The Animation
5,anime,27,24,Haruka Nogizaka's Secret
6,manga,33,24,Ai non stop!
7,anime,162,88,Dragon Ball GT
8,anime,67,55,Dog Days
9,anime,37,23,Glasslip


## Most controversial

In [24]:
pd.DataFrame(list(Work.objects.controversial().values('title', 'nb_likes', 'nb_dislikes')[:10]))

,nb_dislikes,nb_likes,title
0,161,182,School Days
1,109,125,Dragon Ball Z Movie 11: Bio-Broly
2,58,59,Rail Wars!
3,67,74,Freezing
4,66,77,Chaos;Head
5,58,52,To Love
6,96,124,Kiss x Sis
7,52,48,Sakura Trick
8,91,117,Naruto the Movie 2: Legend of the Stone of Gelel
9,158,120,Yu-Gi-Oh! GX


## Pearls

In [25]:
pd.DataFrame(list(Work.objects.pearls().values('title', 'nb_likes', 'nb_dislikes')[:10]))

,nb_dislikes,nb_likes,title
0,0,19,Owari no Seraph: Owaranai Seraph
1,0,22,GARO THE ANIMATION
2,0,34,The Boy and the Beast
3,0,37,Psycho-Pass: The Movie
4,1,56,Kara no Kyoukai 6: Boukyaku Rokuon
5,1,45,Samurai X: Trust and Betrayal
6,1,42,A Letter to Momo
7,1,42,My Teen Romantic Comedy SNAFU TOO!
8,1,41,True Tears
9,1,40,Tasogare Otome x Amnesia: Taima Otome


## Random

In [26]:
pd.DataFrame(list(Work.objects.random().order_by('?').values('title', 'nb_likes', 'nb_dislikes')[:5]))

,nb_dislikes,nb_likes,title
0,14,49,Ground Control to Psychoelectric Girl
1,15,172,The Tale of The Princess Kaguya
2,1,20,Kannagi: Moshimo Kannagi ga Attara...
3,4,27,Switch girl
4,9,82,Ghost in the Shell 2: Innocence


## I said random

In [27]:
pd.DataFrame(list(Work.objects.random().order_by('?').values('title', 'nb_likes', 'nb_dislikes')[:5]))

,nb_dislikes,nb_likes,title
0,2,24,Air in Summer
1,4,26,Reset
2,9,79,Lucky☆Star: Original na Visual to Animation
3,16,257,Fate/stay night: Unlimited Blade Works (TV)
4,4,22,Toaru Majutsu no Index Specials


## Special selection

In [28]:
selection = [
    Work.objects.get(title='Kaiba').id,
    Work.objects.get(title='Madlax').id,
    Work.objects.filter(title__icontains='Fujiko')[3].id,
    902
]

In [29]:
pd.DataFrame(list(Work.objects.filter(id__in=selection).values('title', 'nb_likes', 'nb_dislikes')))

,nb_dislikes,nb_likes,title
0,4,33,Kaiba
1,1,42,A Letter to Momo
2,4,19,Madlax
3,1,14,"Lupin the Third, The Woman Called Fujiko Mine"


## [New!] Interactive pearls

In [33]:
from ipywidgets import interact, interactive
from IPython.display import display

In [39]:
from collections import Counter

nb_likes = Counter()
nb_dislikes = Counter()
for rating in Rating.objects.exclude(choice__in=['willsee', 'wontsee']):
    if rating.choice == 'like':
        nb_likes[rating.work_id] += 1
    elif rating.choice ==  'favorite':
        nb_likes[rating.work_id] += 2
    elif rating.choice == 'neutral':
        nb_dislikes[rating.work_id] += 0.5
    elif rating.choice == 'dislike':
        nb_dislikes[rating.work_id] += 1

def pearls(max_ratings=126, min_ratings=20):
    ranking = Counter()
    for work_id in nb_likes:
        if min_ratings <= nb_likes[work_id] + nb_dislikes[work_id] <= max_ratings:
            dislike_rate = nb_dislikes[work_id] / nb_likes[work_id]
            ranking[work_id] = -dislike_rate
    rank = 1
    lines = []
    for work_id, dislike_rate in ranking.most_common(10):
        lines.append('%d. %s %d likes %d likes' % (rank, Work.objects.get(id=work_id).title, nb_likes[work_id], nb_dislikes[work_id]))
        rank += 1
    print('\n'.join(lines))

In [40]:
display(interactive(pearls))